# Notes from Doug

If people can take one lesson away from reading this, it’s why do we use GPUs and how is the computational model associated with GPUs more sensible for NN needs? 
Sections 5-7 are the real important ones
What parts of NN are able to be parallelized? SIMD, sending batches


Set the stage for GPUs: 
What are they, how do they work? 
Why are they good for matrix multiplication?
How is that useful for machine learning? 
How can convolution/recurrent stuff be reshaped to be used in this way? 


Section 8 can be mined for discussion topics 
What are limits and why - not enough data, or not enough computational time?
What key advances will be necessary to use larger data sets? 
OR what will make training high-performance models available to someone on their desktop? 

# 5 Deep Neural Networks(DNN)

Current trend if toward concurrency within mini-batches
 
Even if SIMD is not explicitly utilized, your code is almost garunteed to use SIMD

GPUs accel at performing matrix and vector operations 

Three periods in the history of classification neural networks:
  1. Experimentation(~1985-2010) 
  2. Growth(2010-2015)
  3. Resource Conservation(2015-present) 

# 6 Concurrency In Networks

## 6.1 Data Parallelism

Pattern Parallelism
  Partition *N* samples into minibatches
  Distrubute batches across computational resources
  
Batch Normalization(BN)
  Can normalize within minibatches
  Or, use Weight Normalization(WN)

## 6.2 Model Parallelism

In case of DNNs, partition work according to dimension (i.e. C, H or W for a 4D tensor). The dependencies which in turn generate communication determines overall performance. Fully connected layers incur all-to-all. Has been proposed to mitigate this problem by making processor responsible for 2x number of nodes. Therefore more computation but less communicantion

In case of CNNs, not as easy. Can use LCNs instead

TODO: exapand on this

TODO: tree nets?

TODO: include / make graphics similiar to figure 14

Note: Should probably (briefly)talk about tree nets as they are analagous to ensamble classifiers 


## 6.3 Pipelining

Overlaping communications 

OR

paritioning of DNN based on depth, assigning layers to specific processors

# 7 Concurrency In Training

## 7.1 Model Consistency

## 7.2 Parameter Distribution

## 7.3 Training Distribution

# 8. Optimization

## 8.1 Parameter Search 

## 8.2 Hyperparameter Search